## **Build Smarter AI Apps: Empowering LLMs with LangChain & Gemini**

## Objectives

After completing this lab, you will be able to:

- Use the core features of the LangChain framework, including prompt templates, chains, and agents, relative to enhancing LLM customization and output relevance.

- Explore LangChain's modular approach, which supports dynamic adjustments to prompts and models without extensive code changes.

- Enhance LLM applications by integrating retrieval-augmented generation (RAG) techniques with LangChain. You'll learn how integrating RAG enables greater accuracy and delivers improved contextually-aware responses.


### Installing required libraries

In [4]:
%%capture
# Ensure the core LangChain ecosystem is up to date
!pip install -U --no-cache-dir langchain langchain-community langchain-google-genai

# Essential utilities for RAG (Retrieval Augmented Generation)
!pip install -U pypdf chromadb langchainhub

# Specifically for Gemini's multimodal and generative features
!pip install -U google-generativeai

## Model

A large language model (LLM) serves as the interface for the AI's capabilities. The LLM processes plain text input and generates text output, forming the core functionality needed to complete various tasks. When integrated with LangChain, the LLM becomes a powerful tool, providing the foundational structure necessary for building and deploying sophisticated AI applications.

### Running Locally
If you are running this lab locally, you will need to configure your own API keys. This lab uses the `ModelInference` module from `Gemini`. To configure your own API key, run the code cell below with your key in the `GEMINI_API_KEY` field of `credentials`. **DO NOT** uncomment the `GEMINI_API_KEY` field if you aren't running locally, it will causes errors.

In [8]:
import os
from getpass import getpass
import google.generativeai as genai

# --- Google Gemini Implementation ---
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# --- LangChain Core (The "Orchestration" Layer - Stays the same!) ---
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.messages import HumanMessage, SystemMessage

# Note: LLMChain is being deprecated in favor of LCEL (LangChain Expression Language)
# I will show you how to use the modern pipe (|) syntax below.

if "GEMINI_API_KEY" not in os.environ:
    os.environ["GEMINI_API_KEY"] = getpass("Enter your Gemini API Key: ")


genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
print("Fetching active models...")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        # This will print the exact string needed for langchain
        print(f"👉 Use this ID: {m.name.replace('models/', '')}")

/home/kungdevz/workspaces/ai-app-langchain-gemini/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_3483/3230939870.py:3: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


Fetching active models...
👉 Use this ID: gemini-2.5-flash
👉 Use this ID: gemini-2.5-pro
👉 Use this ID: gemini-2.0-flash-exp
👉 Use this ID: gemini-2.0-flash
👉 Use this ID: gemini-2.0-flash-001
👉 Use this ID: gemini-2.0-flash-exp-image-generation
👉 Use this ID: gemini-2.0-flash-lite-001
👉 Use this ID: gemini-2.0-flash-lite
👉 Use this ID: gemini-2.0-flash-lite-preview-02-05
👉 Use this ID: gemini-2.0-flash-lite-preview
👉 Use this ID: gemini-exp-1206
👉 Use this ID: gemini-2.5-flash-preview-tts
👉 Use this ID: gemini-2.5-pro-preview-tts
👉 Use this ID: gemma-3-1b-it
👉 Use this ID: gemma-3-4b-it
👉 Use this ID: gemma-3-12b-it
👉 Use this ID: gemma-3-27b-it
👉 Use this ID: gemma-3n-e4b-it
👉 Use this ID: gemma-3n-e2b-it
👉 Use this ID: gemini-flash-latest
👉 Use this ID: gemini-flash-lite-latest
👉 Use this ID: gemini-pro-latest
👉 Use this ID: gemini-2.5-flash-lite
👉 Use this ID: gemini-2.5-flash-image-preview
👉 Use this ID: gemini-2.5-flash-image
👉 Use this ID: gemini-2.5-flash-preview-09-2025
👉 Use t

### Your First Gemini Chain (LCEL Style)
Instead of using the older LLMChain, modern AI engineering uses LCEL (LangChain Expression Language). It is more readable and easier to debug.

Let's initialize the model and create a simple chain:

In [9]:
# 1. Initialize the Model (The "Brain")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", 
    temperature=0.7,
    convert_system_message_to_human=True # Helpful for some Gemini versions
)

# 2. Define the Prompt (The "Instructions")
prompt = ChatPromptTemplate.from_template(
    "You are a senior AI Engineer. Explain {topic} to a junior developer in 3 bullet points."
)

# 3. Create the Chain using the Pipe Operator (|)
# This flows: Input -> Prompt -> LLM -> String Output
chain = prompt | llm | StrOutputParser()

# 4. Execute
response = chain.invoke({"topic": "Vector Embeddings"})
print(response)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Here's how I'd explain vector embeddings to a junior developer:

*   **Think of them as "meaningful coordinates" for data:** Instead of representing text, images, or other data as raw characters or pixels, vector embeddings translate them into lists of numbers (vectors) in a high-dimensional space. The key is that similar pieces of data are placed *close together* in this space, while dissimilar ones are far apart.

*   **They capture semantic relationships:** These numerical representations aren't random. They're learned by AI models that understand context, relationships, and nuances. For example, the embedding for "king" will be mathematically closer to "queen" than it is to "banana," because the model has learned the semantic connection between royalty.

*   **They unlock powerful AI capabilities:** By converting complex data into these numerical vectors, we can perform sophisticated tasks like finding similar documents, recommending products, classifying text, or even answering qu